In [15]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gym
import numpy as np
from RL import helpers
from Hack import load
%matplotlib qt5

In [16]:
epex = load.epex().load()
price_array = epex['apx_da_hourly'].values


In [17]:
max_time = 30769#2*24*28
#env = helpers.energy_price_env(price_array, max_time=max_time, window_size=24*2)
env = helpers.energy_price_env(price_array, max_time=max_time, window_size=24)
model = PPO(MlpPolicy, env, verbose=1)
check_env(env, warn=True)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\mgmf4\anaconda3\envs\ml\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [18]:
mean_reward_before_train = helpers.evaluate(model, num_episodes=10, index = epex.index)

Mean reward: 171.6559 +/- 343.1598 	 Num episodes: 10


In [21]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)
model.save("../model_dir/trained_model")

# load model using loaded_model = PPO.load("path_to_model")

-----------------------------
| time/              |      |
|    fps             | 1693 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1137       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00454426 |
|    clip_fraction        | 0.0205     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.08      |
|    explained_variance   | 0.0065     |
|    learning_rate        | 0.0003     |
|    loss                 | 13.8       |
|    n_updates            | 60         |
|    policy_gradient_loss | -0.00955   |
|    value_loss           | 23.8       |
----------------------------------------
-----------------------------------------
| time/   

In [22]:
# Trained Agent, after training
periods = 7*48#4 * 28*24*2 # len(price_array)-1  # 4 * 28*24*2
new_env =  DummyVecEnv([lambda: helpers.energy_price_env(price_array, start_time=max_time, max_time = periods)])
mean_reward_after_train = helpers.evaluate(model, new_env=new_env, num_episodes=10, index=epex.index)

C:\Users\mgmf4\anaconda3\envs\ml\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Mean reward: 90.6314 +/- 261.46832 	 Num episodes: 10
